# Variance Thrashold

In [1]:
from sklearn.feature_selection import VarianceThreshold
X = [[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], [0, 1, 1]]
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(X)


array([[0, 1],
       [1, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 1]])

In [2]:
pd.DataFrame(X)

,0,1,2
0,0,0,1
1,0,1,0
2,1,0,0
3,0,1,1
4,0,1,0
5,0,1,1


# 2. Univariate feature selection

In [7]:
from sklearn.datasets import make_regression, make_classification
from sklearn.feature_selection import SelectKBest, f_regression, f_classif
from sklearn.metrics import mean_squared_error

# Выбор признаков для задачи регрессии

In [4]:
X, y = make_regression(n_samples=10000, n_features=100, n_informative=10, random_state=42)

In [5]:
from sklearn.linear_model import LinearRegression

In [8]:
lr = LinearRegression().fit(X,y)

y_pred = lr.predict(X)

round(mean_squared_error(y, y_pred), 10)

0.0

In [9]:
fs = SelectKBest(score_func=f_regression, k=5)

X_selected = fs.fit_transform(X, y)
X_selected.shape

(10000, 5)

In [10]:
lr = LinearRegression().fit(X,y)

y_pred = lr.predict(X)

round(mean_squared_error(y, y_pred), 10)

0.0

In [11]:
X[:5]

array([[ 1.63460551e+00, -1.96811123e-01,  3.57162586e-01,
         1.44052232e-01, -2.12337854e+00, -9.49600244e-01,
         4.01466858e-01, -1.99704744e+00, -4.46398499e-01,
        -6.72987219e-01,  1.00929496e+00,  2.72849975e+00,
        -5.43408410e-02,  7.58756193e-01,  1.39199158e+00,
        -8.06331219e-01,  4.17080498e-01, -2.19970087e-01,
        -1.90485698e-01,  1.51901957e+00, -7.31260973e-01,
         1.26149899e+00, -9.07380953e-01, -1.53418096e+00,
         9.34717746e-01,  3.52024829e-01, -8.95947004e-01,
         8.31733138e-01, -1.41982531e+00,  9.67429419e-01,
        -2.42137975e+00,  5.17757816e-01,  1.49901207e+00,
        -8.70766376e-01, -3.21587684e-01,  1.01664321e+00,
         9.65573559e-01,  2.16055476e-02,  3.99546841e-01,
         7.71850486e-01, -1.62920904e+00, -7.79544475e-01,
        -7.36065219e-01, -2.75452163e+00, -1.77202783e+00,
         1.81459007e-01, -9.00811741e-01, -1.00593105e-01,
         1.65585562e-03, -2.80869246e-01,  3.79637282e-0

In [12]:
y[:5]

array([-213.26627574,  271.21058765, -292.7857635 ,   58.37607697,
        171.31905301])

Мы получили колонки данных, которые алгоритм выявил как самые важные на основе оригинального таргета.

# Выбор признаков для задачи классификации
## На входе вещ. признаки, на выходе - категориальные

In [13]:
X, y = make_classification(n_samples= 100, n_features=20, n_informative=2)

In [14]:
# X[:5], y[:5]

In [15]:
fs = SelectKBest(score_func=f_classif, k=2)
X_selected = fs.fit_transform(X, y)
print(X_selected.shape)

(100, 2)


## На входе и на выходе категориальные признаки: решаем реальную задачу

Набор данных - breast cancer - https://archive.ics.uci.edu/ml/datasets/Breast+Cancer

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error

In [17]:
df = pd.read_csv('../data/breast_cancer/breast-cancer.data.csv')

In [18]:
X = df.drop('Class', axis=1)
y = df['Class']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)

Train (191, 9) (191,)
Test (95, 9) (95,)


In [42]:
# prepare input data
def prepare_inputs(X_train, X_test):
    oe = OrdinalEncoder()
    oe.fit(pd.concat([X_train,X_test])
    X_train_enc = oe.transform(X_train)
    X_test_enc = oe.transform(X_test)
    return X_train_enc, X_test_enc

SyntaxError: invalid syntax (2969659128.py, line 5)

In [21]:
# prepare target
def prepare_targets(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    return y_train_enc, y_test_enc

In [22]:
# feature selection
def select_features(X_train, y_train, X_test, method, k):
    fs = SelectKBest(score_func=method, k=k)
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

In [23]:
def fit_evaluate_model(X_train, y_train, X_test, y_test):
    model = LogisticRegression(solver='lbfgs')
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    print(classification_report(y_train, y_train_pred, digits=3))
    print(classification_report(y_test, y_test_pred, digits=3))
    return model

In [38]:
X_train.isnull().sum()

age            0
menopause      0
tumor-size     0
inv-nodes      0
node-caps      0
deg-malig      0
breast         0
breast-quad    0
irradiat       0
dtype: int64

In [41]:
X_train

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
213,50-59,premeno,25-29,0-2,no,1,right,left_up,no
200,60-69,ge40,25-29,0-2,no,3,right,left_low,no
48,40-49,premeno,20-24,0-2,no,1,right,right_up,no
261,40-49,ge40,25-29,12-14,yes,3,left,right_low,yes
279,50-59,ge40,40-44,6-8,yes,3,left,left_low,yes
...,...,...,...,...,...,...,...,...,...
203,50-59,ge40,35-39,0-2,no,2,left,left_low,no
255,40-49,premeno,30-34,6-8,yes,3,right,left_up,no
72,60-69,ge40,20-24,0-2,no,2,left,left_up,no
235,50-59,premeno,25-29,3-5,yes,3,left,left_low,yes


In [40]:
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)

ValueError: Found unknown categories ['24-26'] in column 3 during transform

In [33]:
y_train_enc

NameError: name 'y_train_enc' is not defined

Построим модель используя все признаки.

In [26]:
model = fit_evaluate_model(X_train_enc, y_train_enc,X_test_enc,y_test_enc)

NameError: name 'X_train_enc' is not defined

### Chi2 критерий

Отбираем признаки с помощью критерия Хи2.

In [29]:
X_train_fs, X_test_fs, fs = select_features(X_train_enc, y_train_enc, X_test_enc, method=chi2, k='all')

for i in range(len(fs.scores_)):
    print('Feature %d: %f' % (i, fs.scores_[i]))

plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
plt.show()

NameError: name 'X_train_enc' is not defined

Построим модель на топ признаках полученных с помощью критерия Хи2.

In [30]:
X_train_fs, X_test_fs, fs = select_features(X_train_enc, y_train_enc, X_test_enc, method=chi2, k=4)

NameError: name 'X_train_enc' is not defined

In [31]:
model = fit_evaluate_model(X_train_fs, y_train_enc, X_test_fs, y_test_enc)

NameError: name 'X_train_fs' is not defined

Модель показывает хуже результат на трейне, но лучше на тесте, значит она стала лучше генерализировать.

### Метод mutual information 

Отбираем признаки с помощью mutual information метода. Ваши результаты могут отличаться из-за стохастической природы алгоритма. Целесообразно запустить метод несколько раз и вывести средний результат важности признаков.

In [ ]:
X_train_fs, X_test_fs, fs = select_features(X_train_enc, y_train_enc, X_test_enc, method=mutual_info_classif, k='all')

for i in range(len(fs.scores_)):
    print('Feature %d: %f' % (i, fs.scores_[i]))

plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
plt.show()


In [ ]:
X_train_fs, X_test_fs, fs = select_features(X_train_enc, y_train_enc, X_test_enc, method=mutual_info_classif, k=4)

In [ ]:
model = fit_evaluate_model(X_train_fs, y_train_enc, X_test_fs, y_test_enc)

Чтобы убедиться в том, какой алгоритм действительно даёт наилучший эффект, стоит повторить каждый эксперимент несколько раз и сравнить среднюю производительность.   
Также может быть хорошей идеей изучить использование k-кратной перекрестной проверки вместо простого разделения на трен. / вал. выборки.

# RFE

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV

# Классификация с тремя информативными признаками
X, y = make_classification(
    n_samples=1000,
    n_features=25,
    n_informative=3,
    n_redundant=2,
    n_repeated=0,
    n_classes=8,
    n_clusters_per_class=1,
    random_state=0,
)

In [ ]:
# Создаём RFE объект
svc = LogisticRegression()

min_features_to_select = 1  # Минимальное возможное количество признаков
rfecv = RFECV(
    estimator=svc,
    step=1,
    cv=StratifiedKFold(2),
    scoring="accuracy",
    min_features_to_select=min_features_to_select,
)
rfecv.fit(X, y)

In [ ]:
print("Optimal number of features : %d" % rfecv.n_features_)

In [ ]:
# Выведем количество признаков и соответствующие баллы перекрестной проверки
plt.figure(figsize=(10,6))
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (accuracy)")
plt.plot(
    range(min_features_to_select, len(rfecv.grid_scores_) + min_features_to_select),
    rfecv.grid_scores_,
)
plt.show()

# Model-based и последовательный отбор признаков

In [ ]:
from sklearn.datasets import load_diabetes
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LassoCV

In [ ]:
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

In [ ]:
print(diabetes.DESCR)

## Важность признаков из коэффициентов модели

In [ ]:
lasso = LassoCV().fit(X, y)
importance = np.abs(lasso.coef_)
feature_names = np.array(diabetes.feature_names)
plt.bar(height=importance, x=feature_names)
plt.title("Feature importances via coefficients")
plt.show()

## Отбор признаков на основе их важности

In [ ]:
from sklearn.feature_selection import SelectFromModel
from time import time

threshold = np.sort(importance)[-3] + 0.01

In [ ]:
threshold

In [ ]:
tic = time()
sfm = SelectFromModel(lasso, threshold=threshold).fit(X, y)
toc = time()
print(f"Features selected by SelectFromModel: {feature_names[sfm.get_support()]}")
print(f"Done in {toc - tic:.3f}s")

## Отбор признаков c помощью последовательного отбора признаков

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

tic_fwd = time()
sfs_forward = SequentialFeatureSelector(
    lasso, n_features_to_select=2, direction="forward"
).fit(X, y)
toc_fwd = time()

tic_bwd = time()
sfs_backward = SequentialFeatureSelector(
    lasso, n_features_to_select=2, direction="backward"
).fit(X, y)
toc_bwd = time()

print(
    "Features selected by forward sequential selection: "
    f"{feature_names[sfs_forward.get_support()]}"
)
print(f"Done in {toc_fwd - tic_fwd:.3f}s")
print(
    "Features selected by backward sequential selection: "
    f"{feature_names[sfs_backward.get_support()]}"
)
print(f"Done in {toc_bwd - tic_bwd:.3f}s")

## Анализ результатов

Интересно, что при прямом и обратном выборе был выбран один и тот же набор функций. В общем случае это не так, и оба метода обычно приводят к разным результатам.

Мы также отмечаем, что признаки, выбранные SFS, отличаются от выбранных по важности признаков: SFS выбирает bmi вместо s1. Это звучит разумно, поскольку bmi соответствует третьей по значимости характеристике по коэффициентам. Это весьма примечательно, учитывая, что SFS вообще не использует коэффициенты.

В заключение отметим, что SelectFromModel значительно быстрее, чем SFS. Действительно, SelectFromModel нужно только один раз подогнать модель, в то время как SFS необходимо провести перекрестную проверку множества различных моделей для каждой итерации. Однако SFS работает с любой моделью, в то время как SelectFromModel требует, чтобы базовый оценщик (модель) предоставлял атрибут coef_ или атрибут feature_importances_. Прямая SFS быстрее, чем обратная SFS, потому что ей нужно выполнить только n_features_to_select = 2 итераций, в то время как обратная SFS должна выполнить n_features - n_features_to_select = 8 итераций.

# PCA

## По шагам

In [ ]:
import pandas as pd

df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/'
                      'machine-learning-databases/wine/wine.data',
                      header=None)
df_wine.head()

In [ ]:
df_wine.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

# split into training and testing sets
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3,
    stratify=y, random_state=0
)
# standardize the features
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

In [ ]:
cov_mat = np.cov(X_train_std.T)
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)

In [ ]:
eigen_vals

In [ ]:
# рассчитаем совокупную сумму объясненных отклонений
tot = sum(eigen_vals)
var_exp = [(i / tot) for i in sorted(eigen_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

# plot explained variances
plt.bar(range(1,14), var_exp, alpha=0.5,
        align='center', label='individual explained variance')
plt.step(range(1,14), cum_var_exp, where='mid',
         label='cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.show()

Выполним трансформацию признаков:

In [ ]:
eigen_pairs = [(np.abs(eigen_vals[i]), eigen_vecs[:, i]) for i in range(len(eigen_vals))]

# Сортируем пары собст. чисел и векторов
eigen_pairs.sort(key=lambda k: k[0], reverse=True)

In [ ]:
# eigen_pairs

In [ ]:
w = np.hstack((eigen_pairs[0][1][:, np.newaxis], eigen_pairs[1][1][:, np.newaxis]))
print('Matrix W:\n', w)

In [ ]:
X_train_std[0]

In [ ]:
X_train_std[0].dot(w)

In [ ]:
X_train_pca = X_train_std.dot(w)

In [ ]:
colors = ['r', 'b', 'g']
markers = ['s', 'x', 'o']
for l, c, m in zip(np.unique(y_train), colors, markers):
    plt.scatter(X_train_pca[y_train==l, 0], 
                X_train_pca[y_train==l, 1], 
                c=c, label=l, marker=m) 
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.legend(loc='lower left')
plt.show()

## Sklearn

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

# intialize pca and logistic regression model
pca = PCA(n_components=2)

# fit and transform data
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)

Сравним качество модели до уменьшения размерности признаков и после:

In [ ]:
fit_evaluate_model(X_train_std, y_train, X_test_std, y_test)

In [ ]:
fit_evaluate_model(X_train_pca, y_train, X_test_pca, y_test)

Мы использовали вместо 13 признаков всего 2 и качество ухудшелось не драматично.

In [ ]:
lr = LogisticRegression()
lr.fit(X_train_pca, y_train)

In [ ]:
from matplotlib.colors import ListedColormap 

def plot_decision_regions(X, y, classifier, resolution=0.02):
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # построим поверхность решения
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # визуализируем классы экземпляров
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], 
                    y=X[y == cl, 1],
                    alpha=0.6, 
                    c=[cmap(idx)],
                    edgecolor='black',
                    marker=markers[idx], 
                    label=cl) # участки принятия решений для обучающего набора


plot_decision_regions(X_train_pca, y_train, classifier=lr)
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.legend(loc='lower left')
plt.show()

In [ ]:
# участки принятия решений для тестового набора
plot_decision_regions(X_test_pca, y_test, classifier=lr)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.legend(loc='lower left')
plt.show()